In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, HDBSCAN, OPTICS
from sklearn.metrics.pairwise import haversine_distances

# Important for conversion of lat-long based distance to meters
RADIUS_OF_EARTH_AT_SPACE_NEEDLE = 6366.512563943 # km

In [54]:
# Helper functions
def extract_yearly_data(df, dirname, base_filename, year_range):
    dirpath = os.path.join(os.getcwd(), dirname)
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
    path = os.path.join(dirpath, base_filename)
    for year in year_range:
        year_df = df.loc[df["Year"] == year]
        year_df.to_csv(path + "_year_" + str(year) + ".csv", index=False)
    return


def meters_to_hav(meters, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    """Converts a distance to haversine distance"""
    hav = meters / (R * 1000)
    return hav


def get_range(df, col):
    min_val = df[col].min()
    max_val = df[col].max()
    return min_val, max_val


def linearly_interpolate(x, old_interval, new_interval):
    a, b = old_interval
    c, d = new_interval
    new_x = c + (x - a) * ((d - c) / b - a)
    return new_x

In [55]:
# Distance metrics I was playing with
def euclidean_dist(x, y):
    return np.sqrt(np.sum((x - y) ** 2))


def haversine_np(lon1, lat1, lon2, lat2, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = R * c
    return km


def linearized_haversine(a, b, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    a_rad = a * (2 * np.pi / 360)
    b_rad = b * (2 * np.pi / 360)
    x_1, y_1 = a_rad
    x_2, y_2 = b_rad
    delta_x = R * np.cos(y_1) * (x_2 - x_1)
    delta_y = R * (y_2 - y_1)
    return np.sqrt(delta_x**2 + delta_y**2)

In [56]:
# Load the crime data
crime_df = pd.read_csv("SPD_Crime_Data__2008-Present.csv")

In [57]:
# Drop the crimes with missing time, category, or spatial data
crime_df = crime_df.dropna(
    axis=0,
    how="any",
    subset=["Offense Start DateTime", "Offense Parent Group", "Longitude", "Latitude"],
    inplace=False,
)

# Filter crimes with wacky position data
crime_df = crime_df.loc[(crime_df["Latitude"] > 47) & (crime_df["Latitude"] < 48) ]
crime_df = crime_df.loc[(crime_df["Longitude"] > -123) & (crime_df["Longitude"] < -122)]

# Drop report information to reduce data size
crime_df = crime_df.drop(
    axis=1, columns=["Report Number", "Report DateTime"], inplace=False
)

# Convert time to datetime column
crime_df["Offense Start DateTime"] = pd.to_datetime(crime_df["Offense Start DateTime"])

# Extract offense year, month, day, and time of day
crime_df['Year'] = crime_df['Offense Start DateTime'].dt.year
crime_df['Month'] = crime_df['Offense Start DateTime'].dt.month.astype(int)
crime_df['Day'] = crime_df['Offense Start DateTime'].dt.dayofweek.astype(int)
crime_df['Time'] = crime_df['Offense Start DateTime'].dt.time

# Filter to crimes since 2008
crime_df = crime_df.loc[crime_df["Year"] >= 2008]

# Express lat and long in radians for haversine metric
crime_df["long_rad"] = crime_df["Longitude"].apply(np.radians)
crime_df["lat_rad"] = crime_df["Latitude"].apply(np.radians)

In [58]:
crime_df.head()

,Offense ID,Offense Start DateTime,Offense End DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,...,MCPP,100 Block Address,Longitude,Latitude,Year,Month,Day,Time,long_rad,lat_rad
0,12605873663,2020-02-05 10:10:00,NaN,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,...,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2020,2,2,10:10:00,-2.136038,0.831639
1,12605598696,2020-02-03 08:00:00,02/04/2020 08:00:00 AM,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,...,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2020,2,0,08:00:00,-2.134946,0.832088
2,12605567653,2020-02-02 20:30:00,02/02/2020 09:30:00 PM,A,PROPERTY,ROBBERY,Robbery,120,N,U,...,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2020,2,6,20:30:00,-2.134530,0.831935
3,12605174036,2020-02-05 01:17:00,02/05/2020 02:21:00 AM,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,...,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2020,2,2,01:17:00,-2.136019,0.831526
4,12605081469,2020-02-05 00:51:21,NaN,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,...,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2020,2,2,00:51:21,-2.135693,0.831862


In [11]:
# Create yearly extracts if you want
extract_yearly_data(crime_df, "yearly_extracts", "SPD_Crime_Data", range(2008, 2025))

In [59]:
# Save cleaned data
crime_df.to_csv("cleaned_SPD_Crime_Data.csv")

In [8]:
# Checking range of lat and long columns
min_long, max_long = get_range(crime_df, "Longitude")
min_lat, max_lat = get_range(crime_df, "Latitude")

In [9]:
x_range = 1000 * haversine_np(min_long, min_lat, max_long, min_lat)
y_range = 1000 * haversine_np(min_long, min_lat, min_long, max_lat)

x_range, y_range

(15724.679469803265, 36253.28281720292)

In [10]:
# A linearized version introduces very little error, if we need it
x_range_lin = 1000 * linearized_haversine(np.array([min_long, min_lat]), np.array([max_long, min_lat]))
y_range_lin = 1000 * linearized_haversine(np.array([min_long, min_lat]), np.array([min_long, max_lat]))

x_range_lin, y_range_lin

(15724.68421289776, 36253.28281720292)

In [60]:
# Test runs on cap hill in 2023
cap_hill_23 = crime_df.loc[
    (crime_df["Year"] == 2023) & (crime_df["MCPP"] == "CAPITOL HILL")
]
cap_hill_23.head()

,Offense ID,Offense Start DateTime,Offense End DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,...,MCPP,100 Block Address,Longitude,Latitude,Year,Month,Day,Time,long_rad,lat_rad
1020342,41318760607,2023-01-18 00:00:00,01/18/2023 11:59:00 PM,A,PERSON,ASSAULT OFFENSES,Intimidation,13C,E,E,...,CAPITOL HILL,6XX BLOCK OF HARVARD AVE E,-122.321983,47.624694,2023,1,2,00:00:00,-2.134921,0.831208
1020395,41315958431,2023-01-30 10:30:00,NaN,B,SOCIETY,TRESPASS OF REAL PROPERTY,Trespass of Real Property,90J,E,C,...,CAPITOL HILL,14XX BLOCK OF E HOWELL ST,-122.313516,47.617626,2023,1,0,10:30:00,-2.134774,0.831084
1020446,41312435583,2023-01-29 21:40:00,01/29/2023 10:12:00 PM,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,E,E,...,CAPITOL HILL,14XX BLOCK OF 11TH AVE,-122.318132,47.613513,2023,1,6,21:40:00,-2.134854,0.831013
1020536,41288788167,2023-01-28 19:20:00,01/29/2023 03:20:00 AM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,E,E,...,CAPITOL HILL,1XX BLOCK OF 10TH AVE E,-122.319530,47.619333,2023,1,5,19:20:00,-2.134879,0.831114
1020545,41288762898,2023-01-28 02:00:00,01/28/2023 02:01:00 AM,A,PROPERTY,LARCENY-THEFT,Theft From Building,23D,E,E,...,CAPITOL HILL,15XX BLOCK OF 11TH AVE,-122.318150,47.614674,2023,1,5,02:00:00,-2.134854,0.831033


In [51]:
# Test precomputing distance matrix---it's fast, which is good
latlong = cap_hill_23[["lat_rad", "long_rad"]].to_numpy()
distances = haversine_distances(latlong)
distances * (RADIUS_OF_EARTH_AT_SPACE_NEEDLE * 1000) # Approx distance in meters between crimes

array([[   0.        , 1009.45075617, 1275.44504381, ...,  343.80890441,
         343.80890441,  341.87047085],
       [1009.45075617,    0.        ,  573.07081297, ..., 1266.67123412,
        1266.67123412, 1114.80616356],
       [1275.44504381,  573.07081297,    0.        , ..., 1599.70320173,
        1599.70320173, 1220.48001123],
       ...,
       [ 343.80890441, 1266.67123412, 1599.70320173, ...,    0.        ,
           0.        ,  618.30479661],
       [ 343.80890441, 1266.67123412, 1599.70320173, ...,    0.        ,
           0.        ,  618.30479661],
       [ 341.87047085, 1114.80616356, 1220.48001123, ...,  618.30479661,
         618.30479661,    0.        ]])

### Grouping of crime types
1. We need to group the crime types somehow. I haven't done it yet though

In [ ]:
# Define some interest groups in the future
assault_group = ["ASSAULT OFFENSES"]
theft_group = ["LARCENY-THEFT", "BURGLARY/BREAKING&ENTERING" ]

### Plan
1. GridSearch---store every labeling in output dataframe for comparison in Tableau

In [157]:
def run_dbscans(X, df, supports, eps_meters=125):
    """DBSCAN grid search -> df + summaries

    Labels stored in output. Write to disk for comparison in Tableau.
    We need to fix epsilon small (100 meters) to constrain cluster size."""
    output_df = df.copy()
    run_summaries = []

    total_crime = len(X)
    eps = meters_to_hav(eps_meters)

    for min_samples in supports:
        print(f"DBSCAN clustering with eps={eps_meters}m, min_samples={min_samples}")
        # Cluster the data and extract the labels
        colname = "db_cluster_labels_eps=" + str(eps_meters) + "_ms=" + str(min_samples)
        db = DBSCAN(eps=eps, min_samples=min_samples, metric="haversine").fit(X)
        labels_db = db.labels_
        # Count clusters and noise
        num_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
        num_noise_ = list(labels_db).count(-1)
        percent_clustered = (total_crime - num_noise_) / total_crime
        print("Estimated number of clusters: %d" % num_clusters_)
        print("Estimated number of noise points: %d" % num_noise_)
        print(f"Estimated percentage of crime captured: {percent_clustered}\n")
        summary = {
            "type": "DBSCAN",
            "model": db,
            "params": (eps_meters, min_samples),
            "num_clusters": num_clusters_,
            "num_noise": num_noise_,
            "percent_clustered": percent_clustered,
        }
        # Save the labels in the output df
        run_summaries.append(summary)
        output_df[colname] = labels_db
    return output_df, run_summaries


def run_optics(X, df, supports, eps_meters=125):
    """OPTICS grid search -> df + summaries

    Labels stored in output. Write to disk for comparison in Tableau.
    Similarly, we need to fix a small neighborhood size to limit size."""
    output_df = df.copy()
    run_summaries = []

    total_crime = len(X)
    eps = meters_to_hav(eps_meters)

    for xi in xis:
        for ms in supports:
            print(f"OPTICS clustering with xi={xi}, min_samples={ms}")
            # Cluster the data and extract the labels
            colname = "op_cluster_labels_xi=" + str(xi) + "_ms=" + str(ms)
            op = OPTICS(eps=xi, min_samples=ms, metric="haversine").fit(X)
            labels_op = op.labels_
            # Count clusters and noise
            n_clusters_ = len(set(labels_op)) - (1 if -1 in labels_op else 0)
            n_noise_ = list(labels_op).count(-1)
            print("Estimated number of clusters: %d" % n_clusters_)
            print("Estimated number of noise points: %d" % n_noise_)
            # Save labels in the output df
            output_df[colname] = labels_op
    return output_df


def run_hdbscans(min_size, cluster_selection):
    """HDBSCAN grid search -> df

    Labels stored in output. Write to disk for comparison in Tableau."""
    pass

In [156]:
X = cap_hill_23[["Latitude", "Longitude"]]

In [158]:
dbscans_df, summaries = run_dbscans(X, cap_hill_23, range(15, 105, 5))
summaries

DBSCAN clustering with eps=100m, min_samples=15
Estimated number of clusters: 66
Estimated number of noise points: 1937
Estimated percentage of crime captured: 0.4906652642650539

DBSCAN clustering with eps=100m, min_samples=20
Estimated number of clusters: 40
Estimated number of noise points: 2373
Estimated percentage of crime captured: 0.3760189324217723

DBSCAN clustering with eps=100m, min_samples=25
Estimated number of clusters: 25
Estimated number of noise points: 2700
Estimated percentage of crime captured: 0.2900341835393111

DBSCAN clustering with eps=100m, min_samples=30
Estimated number of clusters: 19
Estimated number of noise points: 2857
Estimated percentage of crime captured: 0.24875098606363397

DBSCAN clustering with eps=100m, min_samples=35
Estimated number of clusters: 13
Estimated number of noise points: 3051
Estimated percentage of crime captured: 0.1977386273994215

DBSCAN clustering with eps=100m, min_samples=40
Estimated number of clusters: 10
Estimated number o

[{'type': 'DBSCAN',
  'model': DBSCAN(eps=1.570718646914388e-05, metric='haversine', min_samples=15),
  'params': (100, 15),
  'num_clusters': 66,
  'num_noise': 1937,
  'percent_clustered': 0.4906652642650539},
 {'type': 'DBSCAN',
  'model': DBSCAN(eps=1.570718646914388e-05, metric='haversine', min_samples=20),
  'params': (100, 20),
  'num_clusters': 40,
  'num_noise': 2373,
  'percent_clustered': 0.3760189324217723},
 {'type': 'DBSCAN',
  'model': DBSCAN(eps=1.570718646914388e-05, metric='haversine', min_samples=25),
  'params': (100, 25),
  'num_clusters': 25,
  'num_noise': 2700,
  'percent_clustered': 0.2900341835393111},
 {'type': 'DBSCAN',
  'model': DBSCAN(eps=1.570718646914388e-05, metric='haversine', min_samples=30),
  'params': (100, 30),
  'num_clusters': 19,
  'num_noise': 2857,
  'percent_clustered': 0.24875098606363397},
 {'type': 'DBSCAN',
  'model': DBSCAN(eps=1.570718646914388e-05, metric='haversine', min_samples=35),
  'params': (100, 35),
  'num_clusters': 13,
  'n

### Extracting time windows
1. Need to extract endpoints of the form (start_month, start_year), (end_month, end_year)

In [161]:
# Extract time window as a series of month, year intervals
def months_to_year_month(base_year, months):
    year = base_year + int(months / 12)
    month = (months % 12)
    return month, year


def extract_windows(start_year, end_year, length=12, step=6):
    """Generates a sequence of cuts as a 'sliding time window' 
    """
    num_months = (end_year - start_year) * 12
    cut_months = range(1, num_months, step)
    cuts = [months_to_year_month(2008, cut) for cut in cut_months]
    return cuts

# Need to join the endpoints into time ranges
extract_windows(2008, 2023)

[(1, 2008),
 (7, 2008),
 (1, 2009),
 (7, 2009),
 (1, 2010),
 (7, 2010),
 (1, 2011),
 (7, 2011),
 (1, 2012),
 (7, 2012),
 (1, 2013),
 (7, 2013),
 (1, 2014),
 (7, 2014),
 (1, 2015),
 (7, 2015),
 (1, 2016),
 (7, 2016),
 (1, 2017),
 (7, 2017),
 (1, 2018),
 (7, 2018),
 (1, 2019),
 (7, 2019),
 (1, 2020),
 (7, 2020),
 (1, 2021),
 (7, 2021),
 (1, 2022),
 (7, 2022)]

### Initial exploration
1. Because we want micro hotspots, we only really have the density params to work with 

In [ ]:
# Test DBSCAN 
db = DBSCAN(eps=meters_to_hav(125), min_samples=15, metric="haversine").fit(X)
labels_db = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_noise_ = list(labels_db).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print(f"Estimated percentage of crime captured: {(support - n_noise_) / support}")

Estimated number of clusters: 66
Estimated number of noise points: 1937
Estimated percentage of crime captured: 0.4906652642650539


In [165]:
# Test OPTICS
op = OPTICS(min_samples=15, max_eps=meters_to_hav(100), xi=0.5, metric="haversine").fit(X)
labels_op = op.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_op)) - (1 if -1 in labels_op else 0)
n_noise_ = list(labels_op).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print(f"Estimated percentage of crime captured: {(support - n_noise_) / support}")

Estimated number of clusters: 67
Estimated number of noise points: 1950
Estimated percentage of crime captured: 0.4872469103339469


/home/edouas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_optics.py:992: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [166]:
# Test HDBSCAN
hdb = HDBSCAN(
    min_cluster_size=15,
    # min_samples=30,
    cluster_selection_epsilon=meters_to_hav(100),
    metric="haversine",
    store_centers="centroid",
).fit(X)
labels_hdb = hdb.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0)
n_noise_ = list(labels_hdb).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 90
Estimated number of noise points: 1193
Estimated percentage of crime captured: 0.6863002892453326
